In [ ]:
import datapane as dp

In [ ]:
import altair as alt
from vega_datasets import data

df = data.iris()
fig = (
    alt.Chart(df)
    .mark_point()
    .encode(x="petalLength:Q", y="petalWidth:Q", color="species:N")
)

In [ ]:
from sqlalchemy import (
    Column,
    DateTime,
    Integer,
    String,
    create_engine,
    desc,
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

Base = declarative_base()
session = None


class Comment(Base):
    __tablename__ = "comments"

    id = Column(Integer, primary_key=True)
    comments_name = Column(String)
    content = Column(String)
    created = Column(DateTime, default=func.now())

    def __init__(self, comments_name, content):
        self.comments_name = comments_name
        self.content = content


def db_connect():
    engine = create_engine(
        "sqlite:///./comments.db", connect_args={"check_same_thread": False}
    )

    Base.metadata.create_all(engine)
    global session
    Session = sessionmaker(engine)
    session = Session()


def get_comments(name):
    comments = (
        session.query(Comment)
        .filter_by(comments_name=name)
        .order_by(Comment.created.desc())
        .all()
    )

    if comments:
        blocks = [dp.Text(comment.content) for comment in comments]
    else:
        blocks = [dp.Empty(name=f"{name}_no_comments")]

    return dp.Group(blocks=blocks, name=f"{name}_comments")


def post_comment(params, name):
    session.add(Comment(comments_name=name, content=params["comment"]))
    session.commit()
    return get_comments(name)


def with_comments(block, name):
    import functools

    post_comments_partial = functools.partial(post_comment, name=name)

    comments_block = dp.Group(
        blocks=[
            dp.Function(
                post_comments_partial,
                target=f"{name}_comments",
                controls=dp.Controls(dp.TextBox(f"comment", "Post a comment")),
            ),
            get_comments(name),
        ],
        label="Comments",
    )

    block = dp.Group(block, label=block._tag)
    block_with_comments = dp.Select(
        blocks=[block, comments_block], type=dp.SelectType.TABS
    )

    return block_with_comments


db_connect()

In [ ]:
def app():
    return dp.View(
        dp.Group(
            with_comments(dp.Plot(fig), "iris_plot"),
            with_comments(dp.DataTable(df), "iris_df"),
            columns=2,
        )
    )

In [ ]:
dp.serve(app)